In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [40]:
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 15, 10
rcParams["font.weight"] = "bold"
rcParams["axes.labelweight"] = "bold"
rcParams["font.size"] = 12

In [41]:
df_placas = pd.read_csv('placas.csv')

df_placas

FileNotFoundError: [Errno 2] No such file or directory: 'placas.csv'

In [ ]:
df_placas.isna().sum() # Verificando los valores nulos

In [ ]:
df_placas.info() # Verificando los tipos de datos

In [ ]:
df_placas["MODULE_TEMPERATURE"].isna()

In [ ]:
df_placas["MODULE_TEMPERATURE"].isna().sum()

In [ ]:
df_placas["MODULE_TEMPERATURE"][df_placas["MODULE_TEMPERATURE"].isna()]

In [ ]:
df_placas[20:330][["MODULE_TEMPERATURE", "AMBIENT_TEMPERATURE", "IRRADIATION"]]

In [ ]:
df_placas.drop(df_placas.dropna().index)

In [ ]:
# No hay nada que limpiar

df_placas.shape[0] - 0

In [ ]:
df_solar_filt = df_placas.dropna().reset_index(drop=True)
df_solar_filt

### Análisis exploratorio, tratamiento y limpieza de datos


In [ ]:
df_solar_filt.describe()

In [ ]:
# consultar el tipo de datos
df_solar_filt.dtypes

In [ ]:
df_solar_filt.dtypes[df_solar_filt.dtypes == "object"]

In [ ]:

df_solar_filt['DATE_TIME'].value_counts()

In [ ]:
# columnas a eliminar por ser informativas
columnas_drop = ["DATE_TIME", "SOURCE_KEY"]

In [ ]:
df_solar_filt2 = df_solar_filt.drop(columns = columnas_drop)
print("Tamaño del tablón filtrado: ", df_solar_filt.shape)
print("Tamaño del tablón nuevo: ", df_solar_filt2.shape)

In [ ]:
df_solar_filt2.dtypes.value_counts()

### Reducción de variables
Importancia de variables

In [ ]:
# generar la lista de features y la variable target
target = 'MODULE_TEMPERATURE'
features = [x for x in df_solar_filt2.columns if x != target]

print(target)
print(features)

In [ ]:
df_solar_filt2[target]

In [ ]:
# Importar el algoritmo de árboles de decisión
from sklearn.tree import DecisionTreeRegressor

# Asignar el algortimo e indicar la profundidad máxima del árbol (con un número rotandamente grande para sobreajustar)
arbol_importancia = DecisionTreeRegressor(max_depth=len(features)+10, random_state=42)

# Entrenar un árbol con todo el conjunto de datos
arbol_importancia.fit(X=df_solar_filt2[features], y=df_solar_filt2[target])

In [ ]:
# Comprobamos que se obtiene un R^2 muy alto. Lo desesable es que sea 1.
y_pred_arbol = arbol_importancia.predict(X=df_solar_filt2[features])

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas para evaluar la calidad del modelo
print('Mean Absolute Error:', mean_absolute_error(df_solar_filt2[target], y_pred_arbol))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_solar_filt2[target], y_pred_arbol)*100)
print('Mean Squared Error:', mean_squared_error(df_solar_filt2[target], y_pred_arbol))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(df_solar_filt2[target], y_pred_arbol)))
print('R^2 coefficient of determination:', r2_score(df_solar_filt2[target], y_pred_arbol))

In [ ]:
# Importancia de cada variable en el árbol ajustado
arbol_importancia.feature_importances_

In [ ]:
# Crear un dataframe con los datos de importancia
importancia = pd.DataFrame(arbol_importancia.feature_importances_, index=features, columns=["Importancia"])

# Ordenamos los datos
importancia.sort_values(by=importancia.columns[0], ascending=False, inplace=True)
importancia

In [ ]:
importancia["imp_acum"] = importancia["Importancia"].cumsum()
importancia

In [ ]:
# Conjunto de variables más importantes
importancia.loc[importancia['imp_acum']<=0.90]

In [ ]:
variables = importancia.loc[importancia['imp_acum']>0.90].index.to_list()
print(variables)

Todas las variables tienen importancia, no eliminamos ninguna.

### Planteamiento del ejercicio de clasificación

# Hacemos una copia del tablón filtrado para no trabajar sobre original


In [ ]:
# Hacemos una copia del tablón filtrado para no trabajar sobre original
df_solpl = df_solar_filt2.copy()

In [ ]:
#Crearemos ahora una nueva columna llamada escenario
df_solpl["escenario"] = np.where(df_solpl["MODULE_TEMPERATURE"]<df_solpl["MODULE_TEMPERATURE"].mean(),"frio","calor")
df_solpl.drop(["MODULE_TEMPERATURE"],axis=1,inplace=True)
df_solpl

In [ ]:
df_solpl.describe()

In [ ]:
#graficar primer escenario
plt.figure(figsize=(10,5))
sns.violinplot(data=df_solpl, orient="h")
plt.show()

### **Paso 1.**  Obtención y preparación de datos